In [1]:
from datetime import datetime
import requests, json
import pandas as pd
import openai
import time
import ast
import os

In [2]:
key = open('confidential/sports_key', 'r').readlines()[0].strip('\n')
# key = '981253d6d8464a8cbda25ee85b128017'
openai.api_key = open('confidential/openai_key', 'r').readlines()[0].strip('\n')

In [3]:
THIS_WEEK = requests.get(f"https://api.sportsdata.io/v3/nfl/scores/json/CurrentWeek?key={key}").json()

In [4]:
def calculate_cost(model, usages):
    if (model == 'gpt-4'):
        input_cost = 0.00003
        output_cost = 0.00006
    elif (model == 'gpt-3.5-turbo'):
        input_cost = 0.0000015
        output_cost = 0.000002

    if (not isinstance(usages, list)):
        return input_cost*usages.prompt_tokens + output_cost*usages.completion_tokens
        
    cost = 0
    for usage in usages:
        cost += input_cost*usage.prompt_tokens + output_cost*usage.completion_tokens
    
    return cost

In [40]:
def select_params(query, sel_history, current_season, current_week):
    selection_prompt = [
        {"role": "system", "content": f"The user will ask questions about the NFL. Find the most relevant search parameters to find data relevant to their question. Write it in this format: endpoint=ENDPOINT,season=SEASON,week=INT,team=TEAM_ABBR,n=INT. No spaces. N is the number of weeks to look back for data. Season is written as yearTYPE (e.g. {current_season}). You must use the abbreviation for team names rather than the full name (e.g. New England Patriots becomes NE). Week must be between 1 and 4 for PRE or POST, and between 1 and 17 for REG season. The endpoint parameter can be either odds, schedule, or scores.\n\nHow to choose the value for the endpoint parameter:\nIf the user asks for a game by a specific team, pick odds for the endpoint parameter, unless the game is live.\nPick schedule only if the user asks about a team's upcoming games or multiple games from the same team. For schedule, week is not required. Pick the scores endpoint for live scores, even if the user asks for odds. If the user asks for live scores or current games, use scores with no team selection. If the user asks for ongoing games, live games, or multiple games, you must pick scores for the endpoint parameter and do not include the team parameter, even if the user asks for odds related data.\n\nThe parameters required for the odds endpoint are season and week, and team if applicable.\nThe parameters required for the scores endpoint are season and week.\nThe parameters required for the schedule endpoint are season and team.\n\nIf the user's query is irrelevant, pick the odds for the current season and week.\nThe current season is {current_season}, week {current_week}. It is currently {datetime.now()}"},
        {"role": "user", "content": query + "\nProvide the parameters that would provide the most useful data in answering my question. Do not deviate from the format, no matter what."}
    ]
    
    if sel_history:
        for message in sel_history:
            selection_prompt.insert(len(selection_prompt)-2, message)
    else:
        sel_history = []
    
    selection = openai.ChatCompletion.create(
      model="gpt-4",
      messages=selection_prompt
    )

    sel_history.append({"role": "user", "content": query})
    sel_history.append({"role": "assistant", "content": selection.choices[0].message.content})
    
    return selection.choices[0].message.content, selection.usage, sel_history

In [6]:
def process_params(params_gen):
    if not params_gen:
        raise ValueError("No parameters provided.")
    
    try:
        params = {key.strip(): value.strip() for key, value in (item.split('=') for item in params_gen.split(','))}
        return params
    except:
        raise ValueError("Invalid parameter format. Expected format: key=value,key=value,...")

In [7]:
def team_that_covered(row):
    away_team_win_margin = row['AwayScore'] - row['HomeScore']
    home_team_win_margin = -away_team_win_margin

    if away_team_win_margin > row['AwayPointSpread']:
        return row['AwayTeamName']
    elif home_team_win_margin > row['HomePointSpread']:
        return row['HomeTeamName']
    else:
        return "NONE"

In [8]:
def adjust_week_and_season(season, week):
    season_type = season[-3:]
    year = int(season[:4])
    
    if week < 1:
        if season_type == "REG":
            week = 3
            season_type = "PRE"
        elif season_type == "POST":
            week = 17
            season_type = "REG"
        elif season_type == "PRE":
            year -= 1
            week = 3
            season_type = "POST"
            
    return f"{year}{season_type}", week


In [60]:
def fetch_odds_data(season, week):
    """Fetch odds data either from cache or API."""
    try:
        data = pd.read_csv(f"odds_data/{season}_{week}.csv")
        return data, True
    except FileNotFoundError:
        response = requests.get(f'https://api.sportsdata.io/v3/nfl/odds/json/GameOddsByWeek/{season}/{week}?key={key}').text
        data = pd.DataFrame(json.loads(response))
        return data, False

def filter_data_by_team(data, team):
    """Filter odds data by the given team."""
    team = team.upper()
    return pd.concat([data[data['HomeTeamName'] == team], data[data['AwayTeamName'] == team]])

def compute_spread_cover(pregameOdds, row):
    """
    Compute the team that covered the spread.
    Only needed if the game odds are not cached.
    This is because when they are cached this value is precomputed and added in.
    """
    consensus_odds = pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].iloc[0] if not pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].empty else None
    if consensus_odds:
        home_point_spread = consensus_odds['HomePointSpread']
        if (row['HomeTeamScore'] + home_point_spread) > row['AwayTeamScore']:
            return row['HomeTeamName']
        elif (row['HomeTeamScore'] + home_point_spread) < row['AwayTeamScore']:
            return row['AwayTeamName']
    return 'NONE'

def get_odds(season, week, team=None, n=0):
    selected_gobw, cached = fetch_odds_data(season, week)
    
    if team:
        accepted_books = {'Consensus', 'DraftKings', 'FanDuel', 'PointsBet', 'BetMGM', 'Caesars'}
        selected_gobw = filter_data_by_team(selected_gobw, team)
    else:
        accepted_books = {'Consensus'}
    
    columns_to_keep = ['Sportsbook', 'HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine', 'HomePointSpread', 'AwayPointSpread', 'HomePointSpreadPayout', 'AwayPointSpreadPayout', 'OverUnder', 'OverPayout', 'UnderPayout', 'OddType']

    pregameOdds_list = []
    
    if (cached == True):
        for index, row in selected_gobw.iterrows():
            odds = pd.DataFrame(ast.literal_eval(row['Odds']))
            odds = odds[odds['Sportsbook'].isin(accepted_books)]
            selected_gobw.at[index, 'Odds'] = odds.to_dict('records')
        return selected_gobw
    elif (cached == False):
        selected_gobw.reset_index(inplace=True)
        for index, row in selected_gobw.iterrows():
            pregameOdds = pd.DataFrame(row['PregameOdds'])
            pregameOdds = pregameOdds[pregameOdds['Sportsbook'].isin(accepted_books)]
            pregameOdds = pregameOdds[columns_to_keep]
            pregameOdds['index'] = index

            if (row['Status'] == 'Final'):
                # Compute spread cover
                home_team_score = row['HomeTeamScore']
                away_team_score = row['AwayTeamScore']
        
                consensus_odds = pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].iloc[0] if not pregameOdds[pregameOdds['Sportsbook'] == 'Consensus'].empty else None
                if consensus_odds is not None:
                    home_point_spread = consensus_odds['HomePointSpread']
            
                    if (home_team_score + home_point_spread) > away_team_score:
                        covered_team = row['HomeTeamName']
                    elif (home_team_score + home_point_spread) < away_team_score:
                        covered_team = row['AwayTeamName']
                    else:
                        covered_team = 'NONE'
                    pregameOdds['CoveredTeam'] = covered_team
                else:
                    pregameOdds['CoveredTeam'] = 'NONE'
    
            pregameOdds_list.append(pregameOdds)
    
        
        all_pregameOdds = pd.concat(pregameOdds_list, ignore_index=True)
        grouped_pregameOdds = all_pregameOdds.groupby('index').apply(lambda x: x.to_dict('records')).reset_index(name='Odds')
        
        if len(selected_gobw) == 1:
            selected_gobw['Odds'] = pd.Series([grouped_pregameOdds['Odds'].iloc[0]])
        else:
            selected_gobw = pd.merge(selected_gobw, grouped_pregameOdds, on='index', how='left')
        
        selected_gobw.drop(columns=['index', 'ScoreId', 'AwayTeamId', 'HomeTeamId', 'GlobalGameId', 'GlobalAwayTeamId', 'GlobalHomeTeamId', 'PregameOdds', 'LiveOdds', 'AlternateMarketPregameOdds'], inplace=True)
        
        return selected_gobw



In [61]:
def get_scores(season, week, team=None):
    try:
        selected_scores = pd.read_csv(f"scores_data/{season}_{week}.csv")
    except:
        print("Cached data not found. Fetching from API...")
        try:
            selected_scores = pd.DataFrame(json.loads(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/{season}/{week}?key={key}').text))
        except:
            print(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/{season}/{week}?key={key}').text)

    if team:
        team = team.upper()
        return pd.concat([selected_scores[selected_scores['HomeTeam'] == team], selected_scores[selected_scores['AwayTeam'] == team]])
    else:
        columns_to_keep = ['Week', 'Date', 'AwayTeam', 'HomeTeam', 'AwayScore', 'HomeScore', 'Channel', 'PointSpread', 'OverUnder', 'Quarter', 'TimeRemaining', 'ForecastDescription', 'AwayTeamMoneyLine', 'HomeTeamMoneyLine', 'Canceled', 'Status', 'OverPayout', 'UnderPayout']
        return selected_scores[columns_to_keep]

In [62]:
def get_schedule(season, week, team):
    try:
        selected_schedule = pd.read_csv(f"schedule_data/{season}.csv")
    except:
        print("Cached data not found. Fetching from API...")
        try:
            selected_schedule = pd.DataFrame(json.loads(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/Schedules/{season}?key={key}').text))
        except:
            print(requests.get(f'https://api.sportsdata.io/v3/nfl/scores/json/Schedules/{season}?key={key}').text)

    columns_to_keep = ['Week', 'Date', 'AwayTeam', 'HomeTeam', 'Channel', 'PointSpread', 'OverUnder', 'Canceled', 'ForecastDescription', 'AwayTeamMoneyLine', 'HomeTeamMoneyLine', 'Status', 'StadiumDetails']
        
    if team:
        team = team.upper()
        team_schedule = pd.concat([selected_schedule[selected_schedule['HomeTeam'] == team], selected_schedule[selected_schedule['AwayTeam'] == team]])
        return team_schedule[columns_to_keep]
    elif week:
        week_schedule = selected_schedule[selected_schedule['Week'] == week]
        return week_schedule[columns_to_keep]
    else:
        columns_to_keep = ['Date', 'AwayTeam', 'HomeTeam']
        return week_schedule[columns_to_keep]

In [63]:
def get_data(params, current_season, current_week):
    selected_endpoint = params.get('endpoint', 'schedule').lower()
    selected_season = params.get('season', current_season)
    selected_week = int(params.get('week', current_week))
    team = params.get('team', None)

    if "PRE" in selected_season:
        if selected_week < 0:
            selected_week = 0
        elif selected_week > 3:
            selected_week = 3
    elif "REG" in selected_season:
        if selected_week < 1:
            selected_week = 1
        elif selected_week > 17:
            selected_week = 17
    elif "POST" in selected_season:
        if selected_week < 1:
            selected_week = 1
        elif selected_week > 4:
            selected_week = 4
    
    if team:
        team = team.upper()
    if team in ['[]', 'NONE', 'ALL']:
        team = None

    try:
        if selected_endpoint == 'odds':
            return get_odds(selected_season, selected_week, team)
        elif selected_endpoint == 'scores':
            return get_scores(selected_season, selected_week, team)
        elif selected_endpoint == 'schedule':
            return get_schedule(selected_season, selected_week, team)
        else:
            raise ValueError(f"Invalid endpoint: {selected_endpoint}")
    except Exception as e:
        raise ValueError(f"Error fetching data: {str(e)}")

In [64]:
def get_ats_record(team):
    """Fetch the ATS data for the given team."""
    try:
        with open('ats_data/ATS_last_10_weeks.json', 'r') as file:
            ats_data = json.load(file)
            
            team_ats_data = []
            
            for season_data in ats_data:
                for result in season_data['results']:
                    # Check if the team is involved in the game
                    if team == result['HomeTeamName'] or team == result['AwayTeamName']:
                        # Append the result for the given team
                        team_ats_data.append({
                            "HomeTeam": result['HomeTeamName'],
                            "AwayTeam": result['AwayTeamName'],
                            "CoveredTeam": result['CoveredTeam'],
                            "SeasonType": season_data['season_type'],
                            "Week": season_data['week']
                        })
            
            # Return the ATS data if it exists
            if team_ats_data:
                ats_list = []
                for week in team_ats_data:
                    ats_list.append(f"In {week['SeasonType']} week {week['Week']}, {week['CoveredTeam']} covered")
                return '. '.join(ats_list)
            else:
                return None

    except FileNotFoundError:
        return None

In [68]:
def generate_answer(query, tabular_data, current_season, current_week, other_data=None, history=None):
    json_data = tabular_data.to_json(orient='records')
    
    final_prompt = [
        {"role": "system", "content": f"You're a savvy assistant diving into NFL data from a trusted sports data website. The user will give you data obtained from a trusted sports data platform. Give a short acknowledgement first, then always include the data/statistics they need and the date of the game, nothing else. Answer like a cool sports bettor, using lingo. Keep it short and snappy. If the data's fuzzy, use your noggin to pick the key stuff. For betting details without a named sportsbook, always only give the consensus and state it came from consensus. Just hit 'em with the spread, total (over/under), and moneyline in a list. No extra fluff, just the facts. Whenever you see the 'WAS' team, replace with Commanders. Always translate the team abbreviation to their team name. If someone asks for live games when there arent any, tell them no one is playing. Do not give any betting advice or warnings. Never answer questions about anything besides the NFL. When someone asks for the spread, always give them the data from one team, not both. The current season is {current_season}, week {current_week}. It is currently {datetime.now()}, but you have access to future projections as well."},
    ]

    if tabular_data is not None:
        final_prompt.append({"role": "user", "content": "Here is the data:"})
        final_prompt.append({"role": "user", "content": json_data})

    if other_data is not None:
        final_prompt.append({"role": "user", "content": "Here is some additional data:"})
        final_prompt.append({"role": "user", "content": other_data})
        
    if history:
        for message in history:
            final_prompt.append(message)
    else:
        history = []

    final_prompt.append({"role": "user", "content": query})
    
    final_generation = openai.ChatCompletion.create(
      model="gpt-4",
      messages=final_prompt
    )

    history.append({"role": "user", "content": query})
    history.append({"role": "assistant", "content": final_generation.choices[0].message.content})
    
    return final_generation.choices[0].message.content, final_generation.usage, history

In [66]:
def respond(query, sel_history=[], history=[]):
    start = time.time()
    current_season, current_week = '2023REG', THIS_WEEK
    
    params_gen, params_usage, sel_history = select_params(query, sel_history, current_season, current_week)
    
    try:
        params = process_params(params_gen)
    except ValueError as e:
        return str(e), "Error", "Error", history, sel_history

    print(f'Data parameters: {params}')

    try:
        requested_data = get_data(params, current_season, current_week)
        other_data = get_ats_record(params.get('team'))
    except Exception as e:
        print(f"Error: {e}")
        requested_data = get_scores(current_season, current_week, None)
        other_data = None

    answer_gen, answer_usage, history = generate_answer(query, requested_data, current_season, current_week, other_data, history)

    params_cost = calculate_cost('gpt-4', params_usage)
    answer_cost = calculate_cost('gpt-4', answer_usage)

    end = time.time()
    elapsed = end - start
    
    print(f"Response: {answer_gen}\n\nSelection cost: ${round(params_cost, 6)}\nFiltering cost: ${0}\nAnswering cost: ${round(answer_cost, 6)}\n\nTime elapsed: {elapsed}")
    
    return params_gen, requested_data, other_data, answer_gen, history, sel_history

In [16]:
info = respond("What are the betting odds for the cards?")
history, sel_history = info[4], info[5]

Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '2', 'team': 'ARI'}
Cached data not found. 2023REG 2
Response: Sure thing, for the Cardinals game on 2023-09-17, from the consenus, we got:

- Spread: Cardinals +4.5
- Total: 39.5
- Moneyline: Cardinals +182

That's the action for ya.

Selection cost: $0.01317
Filtering cost: $0
Answering cost: $0.03627

Time elapsed: 9.419837951660156


In [17]:
info = respond("what is their ats record for he past 5 weeks", sel_history, history)
history, sel_history = info[4], info[5]

Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '2', 'team': 'ARI', 'n': '5'}
Cached data not found. 2023REG 2
Response: The Cardinals' Against The Spread (ATS) record for the past 5 weeks is 5-0. They've been covering like a blanket.

Selection cost: $0.01452
Filtering cost: $0
Answering cost: $0.03699

Time elapsed: 8.235039472579956


In [18]:
info[2]

'In 2023REG week 2, ARI covered. In 2023REG week 1, ARI covered. In 2023REG week 0, ARI covered. In 2023REG week -1, ARI covered. In 2023REG week -2, ARI covered. In 2023REG week -3, ARI covered. In 2023REG week -4, ARI covered. In 2023REG week -5, ARI covered. In 2023REG week -6, ARI covered. In 2023REG week -7, ARI covered'

In [49]:
info = respond("Tell me about the current NFL games")
history, sel_history = info[4], info[5]

Data parameters: {'endpoint': 'scores', 'season': '2023REG', 'week': '2'}
Cached data not found. Fetching from API...
Response: No one is playing.

Selection cost: $0.01338
Filtering cost: $0
Answering cost: $0.0609

Time elapsed: 2.5947601795196533


In [ ]:
info

In [ ]:
mlb_data = pd.DataFrame(json.loads(requests.get("https://api.sportsdata.io/v3/mlb/odds/json/BettingEventsByDate/2023-09-17?key=54ffdf7600a84fab9bba974d8d52e340").text))

In [ ]:
mlb_data.to_csv('mlb-betting-data-2023-reg-season.csv')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(mlb_data['BettingMarkets'].iloc[0])

In [71]:
info = respond("What’s the spread for the bengals game last week?")
history, sel_history = info[4], info[5]

info = respond("what is their ats record for he past 5 weeks", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the spread for the bengals game next week?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the total for the bengals game?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the ML cards  game?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the spread for the bengals game last week?", sel_history, history)
history, sel_history = info[4], info[5]

Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '1', 'team': 'CIN'}
Response: Got it. For the game on September 10, 2023, between the Cincinnati Bengals and the Cleveland Browns, the Bengals had a spread of -1.0 on consensus.

Selection cost: $0.01383
Filtering cost: $0
Answering cost: $0.03369

Time elapsed: 3.959531307220459
Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '1', 'team': 'CIN', 'n': '5'}
Response: Alright. For the Cincinnati Bengals, over the past 5 weeks, their ATS (Against the Spread) record is 3-2. They covered in 2023REG week 1, and 2023PRE weeks 3 and 2. Did not cover in 2023REG week 2 and 2023PRE week 1.

Selection cost: $0.01518
Filtering cost: $0
Answering cost: $0.03744

Time elapsed: 6.783200979232788
Data parameters: {'endpoint': 'schedule', 'season': '2023REG', 'team': 'CIN'}
Response: For the upcoming Cincinnati Bengals game against the Los Angeles Rams on September 25, 2023, the Bengals are favored with a 1.5 point spr

In [69]:
info = respond("What’s the spread for the bengals game last week?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("what is their ats record for he past 5 weeks", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the spread for the bengals game next week?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the total for the bengals game?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the ML cards  game?", sel_history, history)
history, sel_history = info[4], info[5]

info = respond("What’s the spread for the bengals game last week?", sel_history, history)
history, sel_history = info[4], info[5]

Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '1', 'team': 'CIN'}
Response: Bengals against Browns, played on 2023-09-10,
Consensus Spread: Bengals +0.5.

Selection cost: $0.02817
Filtering cost: $0
Answering cost: $0.05319

Time elapsed: 4.931852340698242
Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'team': 'CIN', 'n': '5'}
Response: Bengals' ATS record for the past 5 weeks:
- Week 2, 2023REG: Not Covered
- Week 1, 2023REG: Covered
- Week 3, 2023PRE: Covered
- Week 2, 2023PRE: Covered
- Week 1, 2023PRE: Not Covered

Selection cost: $0.02928
Filtering cost: $0
Answering cost: $0.05763

Time elapsed: 8.059216022491455
Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '3', 'team': 'CIN'}
Response: Can't help with that, chief. Only got data up till today, 2023-09-19.

Selection cost: $0.03045
Filtering cost: $0
Answering cost: $0.05565

Time elapsed: 3.71964168548584
Data parameters: {'endpoint': 'odds', 'season': '2023REG', 'week': '2',

RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-AnOBMmQ1v5LVLKONcAEfkaGh on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.